#Dataset preparation

##Import dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/GTA5.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [2]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/cityscapes'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step3a')

In [4]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=7a1f0ece99f8434400708d17b53c6ed99416e6a60761802751f637bb141f591d
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=484497efe14132b5841cebe99b28bd757526917ef0d0fa35e33c05a077d40c11
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


# checkpoint function

In [5]:
import re
import os
import re

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        print(f"Checkpoint directory {checkpoint_dir} does not exist.")
        return None

    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("bisenet_epoch_") and f.endswith(".pt")]
    if not checkpoints:
        print(f"No checkpoints found in {checkpoint_dir}.")
        return None

    # Extract epoch number from filename
    def extract_epoch(fname):
        match = re.search(r"bisenet_epoch_(\d+).pt", fname)
        return int(match.group(1)) if match else -1

    # Sort by epoch descending
    checkpoints.sort(key=extract_epoch, reverse=True)

    latest = os.path.join(checkpoint_dir, checkpoints[0])
    print(f" Found latest checkpoint: {latest}")
    return latest


# main

In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch

def main():
    transform = transforms.Compose([
        transforms.Resize((720, 1280)),
        transforms.ToTensor()
    ])

    target_transform = transforms.Compose([
        transforms.Resize((720, 1280), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

    dataset_root = '/content/dataset/GTA5'

    full_dataset = GTA5(root=dataset_root, transform=transform, target_transform=target_transform)
    print(f"Full dataset size: {len(full_dataset)}")

    indices = list(range(len(full_dataset)))

    # Split in train/test
    train_indices, val_indices = train_test_split(indices, test_size=0.30, random_state=42)

    # Subsets
    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)

    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Val dataset size: {len(val_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_classes = 19
    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    context_path = 'resnet18'
    checkpoint_dir = "/content/drive/MyDrive/MLDL_repo/step3a/checkpoints3a"

    if not os.path.exists(checkpoint_dir):
      os.makedirs(checkpoint_dir)

    model = BiSeNet(num_classes=num_classes, context_path=context_path)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_epoch = 0
    start_batch = 0

    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    print("latest_ckpt is:", latest_ckpt)

    if latest_ckpt:
        print(f"Restore from checkpoint: {latest_ckpt}")
        checkpoint = torch.load(latest_ckpt, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        start_batch = 0
        print(f"Picking up from epoch {start_epoch}")
    else:
        print("No checkpoint found, start training from scratch")


    # Training
    for epoch in range(start_epoch, epochs):
        current_start_batch = start_batch if epoch == start_epoch else 0
        train_one_epoch(model, train_loader, optimizer, base_lr, epoch, epochs, device,
                        checkpoint_dir=checkpoint_dir, start_batch=current_start_batch)


    final_model_path = "/content/drive/MyDrive/MLDL_repo/step3a/final3a.pt"
    torch.save(model.state_dict(), final_model_path)
    print(f" Final model saved in: {final_model_path}")

if __name__ == "__main__":
    main()


Full dataset size: 2500
Train dataset size: 1750
Val dataset size: 750


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 235MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 214MB/s]


Output streaming troncato alle ultime 5000 righe.
Loss: 0.0506223626434803

 Epoch 49/50 - Batch 87/875
Loss: 0.07128867506980896

 Epoch 49/50 - Batch 88/875
Loss: 0.09164182841777802

 Epoch 49/50 - Batch 89/875
Loss: 0.0628529042005539

 Epoch 49/50 - Batch 90/875
Loss: 0.06086326763033867

 Epoch 49/50 - Batch 91/875
Loss: 0.0591939315199852

 Epoch 49/50 - Batch 92/875
Loss: 0.07361908257007599

 Epoch 49/50 - Batch 93/875
Loss: 0.08927159756422043

 Epoch 49/50 - Batch 94/875
Loss: 0.06586003303527832

 Epoch 49/50 - Batch 95/875
Loss: 0.08282994478940964

 Epoch 49/50 - Batch 96/875
Loss: 0.07187683135271072

 Epoch 49/50 - Batch 97/875
Loss: 0.0800045058131218

 Epoch 49/50 - Batch 98/875
Loss: 0.05862918496131897

 Epoch 49/50 - Batch 99/875
Loss: 0.06907325237989426

 Epoch 49/50 - Batch 100/875
Loss: 0.051266077905893326

 Epoch 49/50 - Batch 101/875
Loss: 0.05716681852936745

 Epoch 49/50 - Batch 102/875
Loss: 0.07377934455871582

 Epoch 49/50 - Batch 103/875
Loss: 0.062331

# validation

In [3]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from datasets_custom.cityscapes import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch
from train import validate
from datasets_custom.labels import GTA5Labels_TaskCV2017


# initialize model
num_classes = 19
context_path = 'resnet18'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BiSeNet(num_classes=num_classes, context_path=context_path).to(device)


# load model
model_path = r"/content/drive/MyDrive/MLDL_repo/step3a/final3a.pt"
model.load_state_dict(torch.load(model_path, map_location='cpu'), strict = False )

model.eval() # python built in function

# Dataset definitions and transformation
transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

target_transform = transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

dataset_root = '/content/cityscapes/Cityscapes/Cityspaces'

test_dataset = CityScapes(root=dataset_root, split='val', transform=transform, target_transform=target_transform)
print(f"Test dataset size: {len(test_dataset)}")
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=2)


# results
best_miou, miou, per_class_ious = validate(model, test_loader, num_classes, device, best_miou=0.0)
print("\n Validation Results on Cityscape:")
print(f" - mIoU: {miou:.4f}")
for idx, label in enumerate(GTA5Labels_TaskCV2017.list_):
    print(f"{label.name:>15}: IoU = {per_class_ious[idx]:.4f}")

Loaded 500 images for split: val
Test dataset size: 500


Validating: 100%|██████████| 250/250 [00:29<00:00,  8.45it/s]

 Validation mIoU: 0.1793
 New best mIoU found!

 Validation Results on Cityscape:
 - mIoU: 0.1793
           road: IoU = 0.4356
       sidewalk: IoU = 0.0716
       building: IoU = 0.5931
           wall: IoU = 0.0939
          fence: IoU = 0.0155
           pole: IoU = 0.0001
          light: IoU = 0.0063
           sign: IoU = 0.0013
     vegetation: IoU = 0.7269
        terrain: IoU = 0.0507
            sky: IoU = 0.6231
         person: IoU = 0.2002
          rider: IoU = 0.0111
            car: IoU = 0.5217
          truck: IoU = 0.0171
            bus: IoU = 0.0325
          train: IoU = 0.0002
      motocycle: IoU = 0.0061
        bicycle: IoU = 0.0000
